# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

# Observations


- It would appear that from randomly selected cities, those showing the best weather for my vacation right now are  in Europe, North Africa and more western counties in the Middle East. 
- The heat maps displaying humidity may lack some validitiy as there is not enough datapoints to shade, and therefore it looks like humidity is highest at the city sites and dissipates outwards which is unlikely to be accurate. 
- I think I will go to Lata, Portugal and stay at the Casa Maquia.  

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
clean_weather_data=pd.read_csv("../output_data/clean_Weather_data.csv")
clean_weather_data


,City,Latitude,Longitude,Country,Temperature (C),Humidity (%),Cloudiness (%),windspeed (m/s)
0,Nishihara,26.18,127.76,JP,26.38,94,75,5.70
1,Rocha,-34.48,-54.33,UY,6.36,93,5,2.39
2,Rikitea,-23.12,-134.97,PF,22.95,82,100,1.17
3,Ushuaia,-54.80,-68.30,AR,-4.00,85,0,2.10
4,Codrington,-38.27,141.97,AU,8.95,86,0,3.71
...,...,...,...,...,...,...,...,...
555,Kuito,-12.38,16.93,AO,11.00,57,0,7.70
556,Polunochnoye,60.87,60.42,RU,27.94,57,93,1.05
557,Cañon City,38.44,-105.24,US,20.10,49,1,9.80
558,Chintamani,13.40,78.07,IN,24.99,78,75,8.70


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
gmaps.configure(api_key=g_key)

In [15]:
Coordinates = clean_weather_data[["Latitude", "Longitude"]]
Humidity=clean_weather_data["Humidity (%)"]


In [16]:
figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig =gmaps.figure(layout=figure_layout)

heat_layer=gmaps.heatmap_layer(Coordinates, weights=Humidity, dissipating=False, max_intensity=20, point_radius=2)
fig.add_layer(heat_layer)
fig



Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
ideal_hol = clean_weather_data.loc[clean_weather_data["Temperature (C)"] <35 ]
ideal_hol =ideal_hol.loc[ideal_hol["Temperature (C)"] >23]
ideal_hol = ideal_hol.loc[ideal_hol["Cloudiness (%)"]==0]
ideal_hol = ideal_hol.loc[ideal_hol["windspeed (m/s)"] <10]
ideal_hol

,City,Latitude,Longitude,Country,Temperature (C),Humidity (%),Cloudiness (%),windspeed (m/s)
120,Köse,40.21,39.65,TR,27.00,32,0,2.10
129,Tolga,34.72,5.38,DZ,29.00,42,0,3.10
151,Uíge,-7.61,15.06,AO,25.11,21,0,2.61
165,Lata,40.16,-8.33,PT,27.54,34,0,0.89
214,Port Hedland,-20.32,118.57,AU,27.00,22,0,5.70
276,Tombouctou,20.00,-3.00,ML,34.95,28,0,4.04
299,Aygedzor,38.98,46.21,AM,32.00,21,0,3.10
303,Jalu,29.03,21.55,LY,31.46,29,0,2.53
313,Vostok,46.49,135.88,RU,26.20,59,0,3.07
317,Ust-Maya,60.42,134.53,RU,29.17,39,0,2.44


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [18]:
hotel_df= ideal_hol
hotel_df["Hotel Name"]=""
hotel_df

,City,Latitude,Longitude,Country,Temperature (C),Humidity (%),Cloudiness (%),windspeed (m/s),Hotel Name
120,Köse,40.21,39.65,TR,27.00,32,0,2.10,
129,Tolga,34.72,5.38,DZ,29.00,42,0,3.10,
151,Uíge,-7.61,15.06,AO,25.11,21,0,2.61,
165,Lata,40.16,-8.33,PT,27.54,34,0,0.89,
214,Port Hedland,-20.32,118.57,AU,27.00,22,0,5.70,
276,Tombouctou,20.00,-3.00,ML,34.95,28,0,4.04,
299,Aygedzor,38.98,46.21,AM,32.00,21,0,3.10,
303,Jalu,29.03,21.55,LY,31.46,29,0,2.53,
313,Vostok,46.49,135.88,RU,26.20,59,0,3.07,
317,Ust-Maya,60.42,134.53,RU,29.17,39,0,2.44,


In [19]:

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "types": "lodging",
    "keywords": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():

     # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
        
    #defind city for each for - for print statement
    city=row["City"]
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotels" and our lat/lng
        

 # make request and print url
    try:
        hotel_nearby = requests.get(base_url, params=params).json()
        hotel_name=hotel_nearby["results"][0]["name"]
        hotel_df.loc[index, 'Hotel Name']=hotel_name
        
        print(f"Retrieving hotel for {city}" )
          
    except:
        
        print(f"No hotel found for {city}")

Retrieving hotel for Köse
Retrieving hotel for Tolga
Retrieving hotel for Uíge
Retrieving hotel for Lata
Retrieving hotel for Port Hedland
No hotel found for Tombouctou
No hotel found for Aygedzor
Retrieving hotel for Jalu
No hotel found for Vostok
No hotel found for Ust-Maya
Retrieving hotel for Bambanglipuro
Retrieving hotel for Dingli
Retrieving hotel for Luchegorsk
Retrieving hotel for Sovetskiy
Retrieving hotel for Srandakan
Retrieving hotel for Sinnai
No hotel found for Abu Samrah
No hotel found for Tessalit
Retrieving hotel for Khovd
Retrieving hotel for Bilma
Retrieving hotel for Kiryat Gat
No hotel found for Argayash


In [20]:
hotel_df

,City,Latitude,Longitude,Country,Temperature (C),Humidity (%),Cloudiness (%),windspeed (m/s),Hotel Name
120,Köse,40.21,39.65,TR,27.00,32,0,2.10,Hicret Oteli
129,Tolga,34.72,5.38,DZ,29.00,42,0,3.10,مشغل جويدة للخياطة
151,Uíge,-7.61,15.06,AO,25.11,21,0,2.61,Juliao Pereira
165,Lata,40.16,-8.33,PT,27.54,34,0,0.89,Casa Maquia
214,Port Hedland,-20.32,118.57,AU,27.00,22,0,5.70,The Esplanade Hotel
276,Tombouctou,20.00,-3.00,ML,34.95,28,0,4.04,
299,Aygedzor,38.98,46.21,AM,32.00,21,0,3.10,
303,Jalu,29.03,21.55,LY,31.46,29,0,2.53,Jalu Hotel
313,Vostok,46.49,135.88,RU,26.20,59,0,3.07,
317,Ust-Maya,60.42,134.53,RU,29.17,39,0,2.44,


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [22]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…